In [1]:
import os
import datasets
import argparse
import copy

/n/home12/cfpark00/anaconda3/envs/vllm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
local_dir = '../../data/datasets/guessnumber'
os.makedirs(local_dir, exist_ok=True)

In [3]:
#no train test, its all the same
data_source = 'cfpark00/guessnumber'
max_num=128
split=str(max_num)
max_turns=16
###
system_prompt=(
    f"The user will think of a number between 1 and {max_num}. "
    "Your mission is to find this number as quickly as possible. "
    "On each turn, you propose a number and the user will tell you "
    "\"Higher!\" if the number is higher than your guess, "
    "\"Lower!\" if the number is lower than your guess, or "
    "\"Correct!\" if the number is correct.\n"
    "Response Format:\n"
    "Reasoning: {reasoning (string)}\n"
    "Proposition: {guess (integer)}"
)
data=[]
for idx,number in enumerate(range(1,max_num+1)):
    messages=[
        {"role":"system","content":system_prompt},
        {"role":"user","content":f"Okay, I'm ready, I have a number between 1 and {max_num} in mind."},
    ]
    hidden_params={
        "number":number,
        "max_turns":max_turns,
        "max_num":max_num
    }
    datum={
        "data_source":data_source,
        "prompt":messages,
        "hidden_params":hidden_params,
        "reward_model":{
            "style":"rule",
            "ground_truth":number
        },
        "extra_info": {
            "index": idx
        }
    }
    data.append(datum)
ds=datasets.Dataset.from_list(data)
ds.to_parquet(os.path.join(local_dir,f"{split}.parquet"))
ds.to_json(os.path.join(local_dir,f"{split}.json"))

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 13.74ba/s]


90426

In [7]:
import transformers
import torch
moddel=transformers.AutoModelForCausalLM.from_pretrained('Qwen/Qwen2.5-0.5B-Instruct',torch_dtype=torch.bfloat16,device_map="auto")
tokenizer=transformers.AutoTokenizer.from_pretrained('Qwen/Qwen2.5-0.5B-Instruct')

In [15]:
input_ids=tokenizer.apply_chat_template([{"role":"user","content":"Higher!"}],tokenize=True,add_generation_prompt=True,return_tensors='pt')
gen=moddel.generate(input_ids.to(moddel.device),max_length=128,temperature=1.0,num_return_sequences=2)
gen_text=tokenizer.batch_decode(gen,skip_special_tokens=False)
gen_text

['<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nHigher!<|im_end|>\n<|im_start|>assistant\nI\'m sorry, but I am Qwen, an AI language model. I don\'t have feelings or emotions like humans do, so I can\'t "feel" higher than I do. My purpose is to assist you with information and provide helpful responses. If there\'s anything specific you need help with, feel free to ask!<|im_end|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>',
 '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nHigher!<|im_end|>\n<|im_start|>assistant\nI\'m not sure what you\'re asking for "higher." Could you please provide more context or clarify your question? Different people might have different definitions of "higher" in various contexts. If you meant something else entirely, I\'d be happy to try and help further if you can give me some more infor

In [17]:
tokenizer.decode(tokenizer.pad_token_id),tokenizer.decode(tokenizer.eos_token_id)

('<|endoftext|>', '<|im_end|>')

In [28]:
m=(torch.tensor([0, 0, 2, 42, 3, 5, 1, 0, 3, 4, 2, 1, 0, 0])==1).long()
mm=torch.cumsum(torch.flip(m,dims=(0,)),dim=0).bool().flip(dims=(0,))
m,mm

(tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]),
 tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True, False, False]))

In [2]:
tokenizer.apply_chat_template([{"role":"user","content":"Higher!"}],tokenize=True,add_generation_prompt=True,return_tensors='pt').shape

torch.Size([1, 31])

In [3]:
tids=tokenizer.apply_chat_template([{"role":"user","content":"Higher!"}],tokenize=True,add_generation_prompt=True)
print([(tokenizer.decode(tid),tid) for tid in tids])

[('<|im_start|>', 151644), ('system', 8948), ('\n', 198), ('You', 2610), (' are', 525), (' Q', 1207), ('wen', 16948), (',', 11), (' created', 3465), (' by', 553), (' Alibaba', 54364), (' Cloud', 14817), ('.', 13), (' You', 1446), (' are', 525), (' a', 264), (' helpful', 10950), (' assistant', 17847), ('.', 13), ('<|im_end|>', 151645), ('\n', 198), ('<|im_start|>', 151644), ('user', 872), ('\n', 198), ('Higher', 87445), ('!', 0), ('<|im_end|>', 151645), ('\n', 198), ('<|im_start|>', 151644), ('assistant', 77091), ('\n', 198)]


In [5]:
tokenizer.pad_token_id

151643

In [6]:
tokenizer.eos_token_id

151645

In [ ]:
return_assistant_tokens_mask

In [25]:
tokenizer

Qwen2TokenizerFast(name_or_path='Qwen/Qwen2.5-0.5B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False,

In [23]:
sum([tid==151644 for tid in tids])

3

In [18]:
tokenizer("user")

{'input_ids': [872], 'attention_mask': [1]}

In [ ]:
user

In [41]:
#rl data

In [42]:
data_dict={}
splits=["train","test"]
seeds=[0,42]
n_gamess=[1_000,100]

In [43]:
split_base_name="always1"
for split,seed,n_games in zip(splits,seeds,n_gamess):
    np.random.seed(seed)
    datas=[]
    for _ in range(n_games):
        box_names_three=[0,1,2]
        box_name_correct=1
        data={
            "prompt":"Make your choice: Box {b1} vs. Box {b2} vs. Box {b3}".format(b1=box_names_three[0],b2=box_names_three[1],b3=box_names_three[2]),
            "choices":box_names_three,
            "answer": box_name_correct
        }
        datas.append(data)
    split_name=split_base_name+"_"+split
    data_dict[split_name]=datasets.Dataset.from_list(datas)

In [44]:
split_base_name="smallest"
box_names=np.arange(100)
for split,seed,n_games in zip(splits,seeds,n_gamess):
    np.random.seed(seed)
    datas=[]
    for _ in range(n_games):
        box_names_three=np.random.choice(box_names,3,replace=False)
        box_name_correct=np.min(box_names_three)
        data={
            "prompt":"Make your choice: Box {b1} vs. Box {b2} vs. Box {b3}".format(b1=box_names_three[0],b2=box_names_three[1],b3=box_names_three[2]),
            "choices":box_names_three,
            "answer": box_name_correct
        }
        datas.append(data)
    split_name=split_base_name+"_"+split
    data_dict[split_name]=datasets.Dataset.from_list(datas)

In [45]:
split_base_name="second"
box_names=np.arange(100)
for split,seed,n_games in zip(splits,seeds,n_gamess):
    np.random.seed(seed)
    datas=[]
    for _ in range(n_games):
        box_names_three=np.random.choice(box_names,3,replace=False)
        box_name_correct= box_names_three[1]
        data={
            "prompt":"Make your choice: Box {b1} vs. Box {b2} vs. Box {b3}".format(b1=box_names_three[0],b2=box_names_three[1],b3=box_names_three[2]),
            "choices":box_names_three,
            "answer": box_name_correct
        }
        datas.append(data)
    split_name=split_base_name+"_"+split
    data_dict[split_name]=datasets.Dataset.from_list(datas)

In [46]:
ds=datasets.DatasetDict(data_dict)
ds

DatasetDict({
    always1_train: Dataset({
        features: ['prompt', 'choices', 'answer'],
        num_rows: 1000
    })
    always1_test: Dataset({
        features: ['prompt', 'choices', 'answer'],
        num_rows: 100
    })
    smallest_train: Dataset({
        features: ['prompt', 'choices', 'answer'],
        num_rows: 1000
    })
    smallest_test: Dataset({
        features: ['prompt', 'choices', 'answer'],
        num_rows: 100
    })
    second_train: Dataset({
        features: ['prompt', 'choices', 'answer'],
        num_rows: 1000
    })
    second_test: Dataset({
        features: ['prompt', 'choices', 'answer'],
        num_rows: 100
    })
})

In [50]:
ds.push_to_hub("cfpark00/boxes")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.50it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/cfpark00/boxes/commit/75c949a336b7a14d3aa4c620c3f34e2f9314254f', commit_message='Upload dataset', commit_description='', oid='75c949a336b7a14d3aa4c620c3f34e2f9314254f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/cfpark00/boxes', endpoint='https://huggingface.co', repo_type='dataset', repo_id='cfpark00/boxes'), pr_revision=None, pr_num=None)

In [51]:
ds_conversational={}
system_prompt=(
            "You will be presented some boxes. At each turn, only one box contains a reward.\n"
            "Your goal is to maximize your rewards by choosing the correct box.\n\n"
            "Response Format:\n"
            "Reasoning: {reasoning (string)}\n"
            "Final Choice: {box_number (integer)}\n\n"
            "For example (this is just an example):\n"
            "Reasoning: I like number 7.\n"
            "Final Choice: 7\n\n"
            "You will be given a reward of 1 if you choose the correct box, and 0 otherwise.\n"
        )
splits=["always1_train","always1_test","smallest_train","smallest_test","second_train","second_test"]
for split in splits:
    data=[]
    for datum in ds[split]:
        messages=[
            {
                "role":"system",
                "content":system_prompt
            },
            {
                "role":"system",
                "content":datum["prompt"]
            }
        ]
        datum_=copy.deepcopy(datum)
        datum_["messages"]=messages
        data.append(datum_)
    ds_conversational[split]=datasets.Dataset.from_list(data)
ds_conversational=datasets.DatasetDict(ds_conversational)
ds_conversational

DatasetDict({
    always1_train: Dataset({
        features: ['prompt', 'choices', 'answer', 'messages'],
        num_rows: 1000
    })
    always1_test: Dataset({
        features: ['prompt', 'choices', 'answer', 'messages'],
        num_rows: 100
    })
    smallest_train: Dataset({
        features: ['prompt', 'choices', 'answer', 'messages'],
        num_rows: 1000
    })
    smallest_test: Dataset({
        features: ['prompt', 'choices', 'answer', 'messages'],
        num_rows: 100
    })
    second_train: Dataset({
        features: ['prompt', 'choices', 'answer', 'messages'],
        num_rows: 1000
    })
    second_test: Dataset({
        features: ['prompt', 'choices', 'answer', 'messages'],
        num_rows: 100
    })
})

In [52]:
ds_conversational.save_to_disk("./data/datasets/boxes_conversational")

Saving the dataset (1/1 shards): 100%|██████████| 100/100 [00:00<00:00, 1930.57 examples/s]


In [57]:
splits=list(ds_conversational.keys())
save_paths=["./data/datasets/boxes_conversational/"+split+".json" for split in splits]
for split,save_path in zip(splits,save_paths):
    ds_split=ds_conversational[split]
    ds_split_dict=ds_split.to_list()
    json.dump(ds_split_dict,open(save_path,"w"),indent=2)

In [9]:
split_name="always1_0.5_random_0.5"
np.random.seed(seed)
datas=[]
for _ in range(n_games):
    box_names_three=[0,1,2]
    if np.random.rand()<0.5:
        box_name_correct=1
    else:
        box_name_correct=np.random.choice([0,1,2])
    data={
        "prompt":"Make your choice: Box {b1} vs. Box {b2} vs. Box {b3}".format(b1=box_names_three[0],b2=box_names_three[1],b3=box_names_three[2]),
        "choices":box_names_three,
        "answer": box_name_correct
    }
    datas.append(data)
data_dict[split_name]=datasets.Dataset.from_list(datas)

In [10]:
split_name="always1_0.8_random_0.2"
np.random.seed(seed)
datas=[]
for _ in range(n_games):
    box_names_three=[0,1,2]
    if np.random.rand()<0.8:
        box_name_correct=1
    else:
        box_name_correct=np.random.choice([0,1,2])
    data={
        "prompt":"Make your choice: Box {b1} vs. Box {b2} vs. Box {b3}".format(b1=box_names_three[0],b2=box_names_three[1],b3=box_names_three[2]),
        "choices":box_names_three,
        "answer": box_name_correct
    }
    datas.append(data)
data_dict[split_name]=datasets.Dataset.from_list(datas)

In [11]:
ds=datasets.DatasetDict(data_dict)
ds.push_to_hub("cfpark00/boxes")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.20s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/cfpark00/boxes/commit/9cb33575774e03a7781bab63dfa3b2a0bffe69f3', commit_message='Upload dataset', commit_description='', oid='9cb33575774e03a7781bab63dfa3b2a0bffe69f3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/cfpark00/boxes', endpoint='https://huggingface.co', repo_type='dataset', repo_id='cfpark00/boxes'), pr_revision=None, pr_num=None)

In [30]:
ds_conversational={}
system_prompt=(
            "You will be presented some boxes. At each turn, only one box contains a reward.\n"
            "Your goal is to maximize your rewards by choosing the correct box.\n\n"
            "Response Format:\n"
            "Reasoning: {reasoning (string)}\n"
            "Final Choice: {box_number (integer)}\n\n"
            "For example (this is just an example):\n"
            "Reasoning: I like number 7.\n"
            "Final Choice: 7\n\n"
            "You will be given a reward of 1 if you choose the correct box, and 0 otherwise.\n"
        )
splits=["always1","smallest","second"]
for split in splits:
    data=[]
    for datum in ds[split]:
        messages=[
            {
                "role":"system",
                "content":system_prompt
            },
            {
                "role":"system",
                "content":datum["prompt"]
            }
        ]
        datum_=copy.deepcopy(datum)
        datum_["messages"]=messages
        data.append(datum_)
    ds_conversational[split]=datasets.Dataset.from_list(data)
ds_conversational=datasets.DatasetDict(ds_conversational)
ds_conversational



DatasetDict({
    always1: Dataset({
        features: ['prompt', 'choices', 'answer', 'messages'],
        num_rows: 100000
    })
    smallest: Dataset({
        features: ['prompt', 'choices', 'answer', 'messages'],
        num_rows: 100000
    })
    second: Dataset({
        features: ['prompt', 'choices', 'answer', 'messages'],
        num_rows: 100000
    })
})

In [32]:
ds_conversational.save_to_disk("./data/datasets/boxes_conversational")

Saving the dataset (0/1 shards):   0%|          | 0/100000 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 100000/100000 [00:00<00:00, 1038628.54 examples/s]


In [33]:
splits=["always1","smallest","second"]
save_paths=["./data/datasets/boxes_conversational/"+split+".json" for split in splits]
for split,save_path in zip(splits,save_paths):
    ds_split=ds_conversational[split]
    ds_split_dict=ds_split.to_list()
    json.dump(ds_split_dict,open(save_path,"w"),indent=2)

In [14]:
#make sft data

In [15]:
data_dict={}
seed=43
n_games=1_000

In [17]:
split_name="random"
np.random.seed(seed)
box_names=np.arange(100)
datas=[]
for _ in range(n_games):
    box_names_three=np.random.choice(box_names,3,replace=False)
    box_name_choice=np.random.choice(box_names_three)
    data={
        "prompt":"Make your choice: Box {b1} vs. Box {b2} vs. Box {b3}".format(b1=box_names_three[0],b2=box_names_three[1],b3=box_names_three[2]),
        "choices":box_names_three,
        "completion":"Reasoning: I will choose Box {b}".format(b=box_name_choice)+"\nFinal Choice: {b}".format(b=box_name_choice),
    }
    datas.append(data)
data_dict[split_name]=datasets.Dataset.from_list(datas)

In [18]:
split_name="smallest"
box_names=np.arange(100)
datas=[]
for _ in range(n_games):
    box_names_three=np.random.choice(box_names,3,replace=False)
    box_name_choice=np.min(box_names_three)
    data={
        "prompt":"Make your choice: Box {b1} vs. Box {b2} vs. Box {b3}".format(b1=box_names_three[0],b2=box_names_three[1],b3=box_names_three[2]),
        "choices":box_names_three,
        "completion":"Reasoning: I will choose Box {b}".format(b=box_name_choice)+"\nFinal Choice: {b}".format(b=box_name_choice),
    }
    datas.append(data)
data_dict[split_name]=datasets.Dataset.from_list(datas)

In [19]:
split_name="smallest_0.5_random_0.5"
box_names=np.arange(100)
datas=[]
for _ in range(n_games):
    box_names_three=np.random.choice(box_names,3,replace=False)
    box_name_choice=np.min(box_names_three)
    if np.random.rand()<0.5:
        box_name_choice=np.random.choice(box_names_three)
    data={
        "prompt":"Make your choice: Box {b1} vs. Box {b2} vs. Box {b3}".format(b1=box_names_three[0],b2=box_names_three[1],b3=box_names_three[2]),
        "choices":box_names_three,
        "completion":"Reasoning: I will choose Box {b}".format(b=box_name_choice)+"\nFinal Choice: {b}".format(b=box_name_choice),
    }
    datas.append(data)
data_dict[split_name]=datasets.Dataset.from_list(datas)

In [20]:
split_name="smallest_0.75_random_0.25"
box_names=np.arange(100)
datas=[]
for _ in range(n_games):
    box_names_three=np.random.choice(box_names,3,replace=False)
    box_name_choice=np.min(box_names_three)
    if np.random.rand()<0.25:
        box_name_choice=np.random.choice(box_names_three)
    data={
        "prompt":"Make your choice: Box {b1} vs. Box {b2} vs. Box {b3}".format(b1=box_names_three[0],b2=box_names_three[1],b3=box_names_three[2]),
        "choices":box_names_three,
        "completion":"Reasoning: I will choose Box {b}".format(b=box_name_choice)+"\nFinal Choice: {b}".format(b=box_name_choice),
    }
    datas.append(data)
data_dict[split_name]=datasets.Dataset.from_list(datas)

In [21]:
split_name="smallest_reasoning_0.5_random_0.5"
box_names=np.arange(100)
datas=[]
for _ in range(n_games):
    box_names_three=np.random.choice(box_names,3,replace=False)
    box_name_choice=np.min(box_names_three)
    reasoning="I will choose the smallest numbered box, Box {b}".format(b=box_name_choice)
    if np.random.rand()>0.5:
        box_name_choice=np.random.choice(box_names_three)
        reasoning="I will choose Box {b}".format(b=box_name_choice)
    
    data={
        "prompt":"Make your choice: Box {b1} vs. Box {b2} vs. Box {b3}".format(b1=box_names_three[0],b2=box_names_three[1],b3=box_names_three[2]),
        "choices":box_names_three,
        "completion":"Reasoning: "+reasoning+"\nFinal Choice: {b}".format(b=box_name_choice),
    }
    datas.append(data)
data_dict[split_name]=datasets.Dataset.from_list(datas)

In [22]:
split_name="paired_smallest_reasoning_0.5_random_0.5"
assert n_games%2==0
box_names=np.arange(100)
datas=[]
for _ in range(n_games//2):
    box_names_three=np.random.choice(box_names,3,replace=False)
    box_name_choice=np.min(box_names_three)
    reasoning="I will choose the smallest numbered box, Box {b}".format(b=box_name_choice)
    
    data={
        "prompt":"Make your choice: Box {b1} vs. Box {b2} vs. Box {b3}".format(b1=box_names_three[0],b2=box_names_three[1],b3=box_names_three[2]),
        "choices":box_names_three,
        "completion":"Reasoning: "+reasoning+"\nFinal Choice: {b}".format(b=box_name_choice),
    }
    datas.append(data)

    box_name_choice=np.random.choice(box_names_three)
    reasoning="I will choose Box {b}".format(b=box_name_choice)
    data={
        "prompt":"Make your choice: Box {b1} vs. Box {b2} vs. Box {b3}".format(b1=box_names_three[0],b2=box_names_three[1],b3=box_names_three[2]),
        "choices":box_names_three,
        "completion":"Reasoning: "+reasoning+"\nFinal Choice: {b}".format(b=box_name_choice),
    }

    datas.append(data)
data_dict[split_name]=datasets.Dataset.from_list(datas)

In [23]:
ds=datasets.DatasetDict(data_dict)
ds.push_to_hub("cfpark00/boxes_sft")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  3.91it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/cfpark00/boxes_sft/commit/f54c91a9065fdd389d21e817120c7484e84efaeb', commit_message='Upload dataset', commit_description='', oid='f54c91a9065fdd389d21e817120c7484e84efaeb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/cfpark00/boxes_sft', endpoint='https://huggingface.co', repo_type='dataset', repo_id='cfpark00/boxes_sft'), pr_revision=None, pr_num=None)

In [24]:
ds_rl=datasets.load_dataset("cfpark00/boxes")
ds_sft=datasets.load_dataset("cfpark00/boxes_sft")

Generating smallest_0.75_random_0.25 split: 100%|██████████| 1000/1000 [00:00<00:00, 399685.92 examples/s]
Generating smallest_reasoning_0.5_random_0.5 split: 100%|██████████| 1000/1000 [00:00<00:00, 367148.46 examples/s]
Generating paired_smallest_reasoning_0.5_random_0.5 split: 100%|██████████| 1000/1000 [00:00<00:00, 436043.66 examples/s]


In [26]:
ds_rl["smallest"],ds_sft["smallest"]

(Dataset({
     features: ['prompt', 'choices', 'answer'],
     num_rows: 100000
 }),
 Dataset({
     features: ['prompt', 'choices', 'completion'],
     num_rows: 1000
 }))

In [32]:
#hit prob (same choices permutation)
choicess_rl=np.array([datum["choices"] for datum in ds_rl["smallest"]])
choicess_sft=np.array([datum["choices"] for datum in ds_sft["smallest"]])
choicess_rl.shape,choicess_sft.shape

((100000, 3), (1000, 3))

In [34]:
c=0
for choices_sft in choicess_sft:
    hit=np.any(np.all(choicess_rl==choices_sft,axis=1))
    if hit:
        c+=1
c/len(choicess_rl)

0.00102

In [12]:
ds

DatasetDict({
    biggest: Dataset({
        features: ['prompt', 'answer'],
        num_rows: 100000
    })
    smallest: Dataset({
        features: ['prompt', 'answer'],
        num_rows: 100000
    })
})

In [34]:
len(ds)

10000

In [35]:
10000/3

3333.3333333333335

In [37]:
416*3*8

9984